In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

draft_data = np.load('draft_data.npy')
print(draft_data)

mmr_column = draft_data[:, 1]  # Extract the second column
# plt.hist(mmr_column, bins=10)  # Adjust the number of bins as needed
# plt.title('Histogram of Second Column')
# plt.xlabel('Values')
# plt.ylabel('Frequency')
# plt.show()


matches_data = []
all_drafts = []
all_wins = []
all_drafts_single_team = []
for match_row in draft_data:
    # print('match_row: ', match_row)
    matches_data.append({
        "match_id": match_row[-1],
        "radiant_win": match_row[0],
        "radiant_team": match_row[2:7],
        "dire_team": match_row[7:12],
        "match_mmr": match_row[1]
    })
    # print(match_row[2:7])
    
    # if match_row[0] == 1:
    #     # all_drafts.append(match_row[2:7])
    #     all_drafts.append(match_row[7:12])
    # else:
    #     all_drafts.append(match_row[2:7])
    #     # all_drafts.append(match_row[7:12])

    #radiant draft entry
    # all_drafts.append(match_row[2:7])
    # if match_row[0] == 1:
    #     all_wins.append(1)
    # else:
    #     all_wins.append(0)
        
    # #dire draft entry
    # all_drafts.append(match_row[7:12])
    # if match_row[0] == 0:
    #     all_wins.append(1)
    # else:
    #     all_wins.append(0)
    
    all_drafts_single_team.append(match_row[2:7])
    all_drafts_single_team.append(match_row[7:12])
        
    #total draft entry
    all_drafts.append(match_row[2:12])
    all_wins.append(match_row[0])
    
# unique_numbers = np.unique(all_drafts)
# print('number of diff heroes: ', len(unique_numbers))
# print(unique_numbers)
all_drafts = pd.DataFrame(all_drafts)
all_drafts_single_team = pd.DataFrame(all_drafts_single_team)


print(matches_data[0])
print(all_drafts)
print('size of all_drafts: ', all_drafts.size)
print('all wins: ', all_wins)
print('all wins size: ', len(all_wins))

In [ ]:
import requests
resp = requests.get("https://api.opendota.com/api/heroes?api_key=96eb7da5-ebee-4944-a6b9-61ca1127a989")
heroes_json = resp.json()
# print(heroes_json)

hero_id_to_name_dict = {}
for hero in heroes_json:
    hero_id_to_name_dict[hero['id']] = hero['localized_name']
print(hero_id_to_name_dict[1])
hero_id_to_name_dict[1] = 'Anti Mage'
print(hero_id_to_name_dict)


# Create the inverse dictionary
name_to_hero_id_dict = {v: k for k, v in hero_id_to_name_dict.items()}

# Print the first few items of the inverse dictionary
for name, hero_id in list(name_to_hero_id_dict.items())[:10]:
    print("Hero Name:", name, "- Hero ID:", hero_id)

def map_values(value):
    return hero_id_to_name_dict.get(value, value)

# Create a new DataFrame with mapped values
all_drafts_names = all_drafts.applymap(lambda x: map_values(x))
print("\nDataFrame with mapped values:")
print(all_drafts_names)

all_drafts_single_team_names = all_drafts_single_team.applymap(lambda x: map_values(x))
print("\nDataFrame with mapped values:")
print(all_drafts_single_team_names)

In [ ]:
# from scipy.linalg import triu
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
print('size of draft df: ', all_drafts_single_team_names[0].size)
# Step 1: Prepare the data (Assuming 'team_drafts' is your DataFrame)
# team_drafts should contain one column for each hero in a team draft
# Each row represents a single team draft, with heroes listed as columns

# Step 2: Train the hero vectors
model = Word2Vec(sentences=all_drafts_single_team_names.values.tolist(), vector_size=50, window=5, min_count=1, sg=1)

# Step 3: Retrieve the hero vectors
hero_vectors_dict = {hero: model.wv[hero] for hero in model.wv.index_to_key}
print('hero vectors dict: ', hero_vectors_dict)

# Step 4: Find similar heroes
def find_similar_heroes(hero, top_n=5):
    similar_heroes = model.wv.most_similar(hero, topn=top_n)
    return similar_heroes

# Step 5: Perform analogies
def perform_analogy(hero_a, hero_b, hero_c, top_n=5):
    try:
        analogy = model.wv.most_similar(positive=[hero_b, hero_c], negative=[hero_a], topn=top_n)
        return analogy
    except KeyError:
        return None

# Example usage:
shadow_fiend_similar_heroes = find_similar_heroes("Shadow Fiend")
print("Similar heroes to Shadow Fiend:")
for hero, similarity in shadow_fiend_similar_heroes:
    print(hero, similarity)
    
shadow_fiend_similar_heroes = find_similar_heroes("Phoenix")
print("\nSimilar heroes to Phoenix:")
for hero, similarity in shadow_fiend_similar_heroes:
    print(hero, similarity)

lion_antimage_analogy = perform_analogy("Lion", "Anti Mage", "Witch Doctor")
print("\nAnalogies:")
for hero, similarity in lion_antimage_analogy:
    print(hero, similarity)


In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Step 1: Extract hero vectors
hero_vectors = np.array([model.wv[hero] for hero in model.wv.index_to_key])
print("hero vectors: ", hero_vectors)

# Step 2: Perform dimensionality reduction using t-SNE
tsne = TSNE(n_components=2, random_state=42)
hero_vectors_2d = tsne.fit_transform(hero_vectors)

#Step 4: Plot the 2D projection of hero vectors with hero names
plt.figure(figsize=(10, 8))
plt.scatter(hero_vectors_2d[:, 0], hero_vectors_2d[:, 1])#, c=colors)

# Annotate each point with hero name
for i, hero in enumerate(model.wv.index_to_key):
    plt.annotate(hero, (hero_vectors_2d[i, 0], hero_vectors_2d[i, 1]), fontsize=6)

plt.title("2-Dimensional Visualization of Hero Vectors from Word2Vec")
plt.xlabel("t-SNE Dimension 1")
plt.ylabel("t-SNE Dimension 2")
plt.grid(True)
plt.show()

In [ ]:
print(hero_vectors[0])

In [ ]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming hero_features is a DataFrame containing the 50 non-concrete features for each hero
# hero_features.shape should be (120, 50)
# hero_features = hero_vectors
hero_names = list(hero_vectors_dict.keys())
hero_vectors = np.array(list(hero_vectors_dict.values()))
# print(hero_names)
# print(hero_vectors)

# Define the number of clusters for each layer (larger numbers mean smaller clusters)
num_clusters_layer1 = 40
num_clusters_layer2 = 30
num_clusters_layer3 = 22
num_clusters_layer4 = 14
num_clusters_layer5 = 9
num_clusters_layer6 = 5

# Perform clustering for each layer
kmeans_layer1 = KMeans(n_clusters=num_clusters_layer1, random_state=3435).fit(hero_vectors)
kmeans_layer2 = KMeans(n_clusters=num_clusters_layer2, random_state=3435).fit(hero_vectors)
kmeans_layer3 = KMeans(n_clusters=num_clusters_layer3, random_state=3435).fit(hero_vectors)
kmeans_layer4 = KMeans(n_clusters=num_clusters_layer4, random_state=3435).fit(hero_vectors)
kmeans_layer5 = KMeans(n_clusters=num_clusters_layer5, random_state=3435).fit(hero_vectors)
kmeans_layer6 = KMeans(n_clusters=num_clusters_layer6, random_state=3435).fit(hero_vectors)

# Get cluster assignments for each hero
hero_cluster_layer1 = kmeans_layer1.labels_
hero_cluster_layer2 = kmeans_layer2.labels_
hero_cluster_layer3 = kmeans_layer3.labels_
hero_cluster_layer4 = kmeans_layer4.labels_
hero_cluster_layer5 = kmeans_layer5.labels_
hero_cluster_layer6 = kmeans_layer6.labels_
clusters = [hero_cluster_layer1, hero_cluster_layer2, hero_cluster_layer3, hero_cluster_layer4, hero_cluster_layer5, hero_cluster_layer6]

# print("cluster layer 1: ", hero_cluster_layer1)
# print("cl1 length: ", len(hero_cluster_layer1))
# print("cluster layer 2: ", hero_cluster_layer2)
# print("cluster layer 3: ", hero_cluster_layer3)

# Create a dictionary mapping each hero name to the clusters they belong to
hero_clusters_dict = {}
for hero_name, cluster1, cluster2, cluster3, cluster4, cluster5, cluster6 in zip(hero_names, hero_cluster_layer1, hero_cluster_layer2, hero_cluster_layer3, hero_cluster_layer4, hero_cluster_layer5, hero_cluster_layer6):
    hero_clusters_dict[hero_name] = [cluster6, cluster5, cluster4, cluster3, cluster2, cluster1]

# Assuming hero_vectors contains vectors for all heroes and clusters contains clusters
phoenix_cluster_level1 = None
phoenix_cluster_level2 = None
phoenix_cluster_level3 = None
phoenix_cluster_level4 = None
phoenix_cluster_level5 = None
phoenix_cluster_level6 = None

# print("hero vectors: ", hero_vectors)
hero_list = model.wv.index_to_key
print("hero list: ", hero_list)
# phoenix_index = np.where(hero_list == "Phoenix")[0]
phoenix_index = hero_list.index("Phoenix")
# print(phoenix_index)
# Assuming clusters are indexed by their level: lowest, middle, highest
level = 1
for cluster in clusters:    
    phoenix_cluster = cluster[phoenix_index]
    indices = [i for i in range(len(cluster)) if cluster[i] == phoenix_cluster]
    # print(indices)
    
    if level == 1:
        phoenix_cluster_1 = indices
        print('\n', phoenix_cluster_1)
    elif level == 2:
        phoenix_cluster_2 = indices
        print(phoenix_cluster_2)
    elif level == 3:
        phoenix_cluster_3 = indices
        print(phoenix_cluster_3)
    elif level == 4:
        phoenix_cluster_4 = indices
        print(phoenix_cluster_4)
    elif level == 5:
        phoenix_cluster_5 = indices
        print(phoenix_cluster_5)
    elif level == 6:
        phoenix_cluster_6 = indices
        print(phoenix_cluster_6,'\n')
    level += 1

def display_loop(phoenix_cluster_x):
    plt.figure(figsize=(10, 8))
    plt.scatter(hero_vectors_2d[:, 0], hero_vectors_2d[:, 1])

    # Annotate each point with hero name
    for i, hero in enumerate(model.wv.index_to_key):
        if hero in [hero_list[j] for j in phoenix_cluster_x]:
            plt.annotate(hero, (hero_vectors_2d[i, 0], hero_vectors_2d[i, 1]), fontsize=6)

    plt.title("2-Dimensional Visualization of Hero Vectors from Word2Vec")
    plt.xlabel("t-SNE Dimension 1")
    plt.ylabel("t-SNE Dimension 2")
    plt.grid(True)
    plt.show()

print("Heroes in Phoenix's 1st layer:",[hero_list[i] for i in phoenix_cluster_1])
display_loop(phoenix_cluster_1)
print("Heroes in Phoenix's 2nd layer:", [hero_list[i] for i in phoenix_cluster_2])
display_loop(phoenix_cluster_2)
print("Heroes in Phoenix's 3rd layer:", [hero_list[i] for i in phoenix_cluster_3])
display_loop(phoenix_cluster_3)
print("Heroes in Phoenix's 4th layer:",[hero_list[i] for i in phoenix_cluster_4])
display_loop(phoenix_cluster_4)
print("Heroes in Phoenix's 5th layer:", [hero_list[i] for i in phoenix_cluster_5])
display_loop(phoenix_cluster_5)
print("Heroes in Phoenix's 6th layer:", [hero_list[i] for i in phoenix_cluster_6])
display_loop(phoenix_cluster_6)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

# Define a function to visualize clusters for a single layer
def visualize_clusters(hero_vectors, cluster_assignments, title):
    # Apply t-SNE to reduce dimensions to 2
    tsne = TSNE(n_components=2, random_state=42)
    tsne_result = tsne.fit_transform(hero_vectors)

    # Get unique cluster assignments
    unique_clusters = np.unique(cluster_assignments)

    # Plot clusters
    plt.figure(figsize=(8, 6))
    for cluster in unique_clusters:
        indices = np.where(cluster_assignments == cluster)[0]
        plt.scatter(tsne_result[indices, 0], tsne_result[indices, 1], label=f'Cluster {cluster}')
        for i, hero in enumerate(model.wv.index_to_key):
            if hero == "Phoenix" or hero == 'Tinker' or hero == 'Shadow Demon' or hero == 'Tusk' or hero == 'Ursa' or hero == 'Dawnreaker' or hero == "Tidehunter" or hero == "Dazzle":
                plt.annotate(hero, (hero_vectors_2d[i, 0], hero_vectors_2d[i, 1]), fontsize=8)
    plt.title(title)
    plt.xlabel('t-SNE Component 1')
    plt.ylabel('t-SNE Component 2')
    plt.legend(fontsize="5")
    plt.grid(True)
    plt.show()

# Visualize clusters for each layer
visualize_clusters(hero_vectors, hero_cluster_layer1, title='Layer 1 Clusters')
visualize_clusters(hero_vectors, hero_cluster_layer2, title='Layer 2 Clusters')
visualize_clusters(hero_vectors, hero_cluster_layer3, title='Layer 3 Clusters')
visualize_clusters(hero_vectors, hero_cluster_layer4, title='Layer 4 Clusters')
visualize_clusters(hero_vectors, hero_cluster_layer5, title='Layer 5 Clusters')
visualize_clusters(hero_vectors, hero_cluster_layer6, title='Layer 6 Clusters')

In [ ]:
print("Sample of Phoenix's hero_vector, 1 closely related hero, 1 slightly related hero, and 1 very different hero")

phoenix_index = hero_list.index("Phoenix")
sd_index = hero_list.index("Shadow Demon")
tusk_index = hero_list.index("Tusk")
spectre_index = hero_list.index("Spectre")
tinker_index = hero_list.index("Tinker")

#record vectors earlier and directly access them for accuracy
phoenix_vector = hero_vectors[phoenix_index]
# print('shape: ', phoenix_vector.shape)
# print(phoenix_vector)
tinker_vector = hero_vectors[tinker_index]
sd_vector = hero_vectors[sd_index]
tusk_vector = hero_vectors[tusk_index]
spectre_vector = hero_vectors[spectre_index]

eight_hero_vectors = np.vstack((phoenix_vector, sd_vector, phoenix_vector, tusk_vector, phoenix_vector, spectre_vector, phoenix_vector, tinker_vector))
hero_names = ['Phoenix', 'Shadow Demon', 'Phoenix', 'Tusk', 'Phoenix', 'Spectre', 'Phoenix', 'Tinker']

# Plot heatmap
plt.figure(figsize=(10, 7))
sns.heatmap(eight_hero_vectors, cmap='viridis', xticklabels=False)
plt.yticks(ticks=np.arange(8) + 0.5, labels=hero_names, rotation=0)
plt.xlabel('NLP Feature Index')
plt.ylabel('Hero')
plt.title('Hero Vectors Comparison')
plt.show()

In [ ]:
pairwise_distances = np.linalg.norm(hero_vectors[:, np.newaxis] - hero_vectors[np.newaxis, :], axis=-1)
np.fill_diagonal(pairwise_distances, np.inf)

min_indices = np.unravel_index(np.argmin(pairwise_distances), pairwise_distances.shape)
closest_pair = min_indices[0], min_indices[1]

# Get the closest vectors
closest_hero1 = hero_list[closest_pair[0]]
closest_hero2 = hero_list[closest_pair[1]]

print("Closest pair of Heroes:")
print("Hero 1:", closest_hero1)
print("Hero 2:", closest_hero2)

hero1_index = min_indices[0]
hero2_index = min_indices[1]

hero1_vector = hero_vectors[hero1_index]
hero2_vector = hero_vectors[hero2_index]

two_hero_vectors = np.vstack((hero1_vector, hero2_vector))
hero_names = [closest_hero1, closest_hero2]

# Plot heatmap
plt.figure(figsize=(10, 1.5))
sns.heatmap(two_hero_vectors, cmap='viridis', xticklabels=False)
plt.yticks(ticks=np.arange(2) + 0.5, labels=hero_names, rotation=0)
plt.xlabel('NLP Feature Index')
plt.ylabel('Hero')
plt.title('Two most similar (perhaps swappable!) heroes')
plt.show()

In [ ]:
#if results are unsatisfactory, I'll switch to predicting off heroes on both teams
#Preparing features
training_data = []
# hero_list = model.wv.index_to_key

# print('all_drafts: ', all_drafts)
# print('all_drafts type: ', type(all_drafts))

for index, row in all_drafts_names.iterrows():
    # print(index)
    draft_list_of_cluster_lists = []
    # print('row: ', row)
    #form the 5x3 array of clusters for each hero
    for hero_name in row:
        # hero_name = hero_list[hero_id]
        
        list_of_clusters = []
        
        #translate hero name back into id
        hero_id = name_to_hero_id_dict[hero_name]
        list_of_clusters.append(hero_id)
        
        # hero_clusters = hero_clusters_dict[hero_name]
        # # print('hero clusters: ', hero_clusters)
        # list_of_clusters.append(hero_clusters[0])
        # list_of_clusters.append(hero_clusters[1])
        # list_of_clusters.append(hero_clusters[2])
        
        hero_clusters = hero_clusters_dict[hero_name]
        print("hero clusters: ", hero_clusters)
        list_of_clusters.extend(hero_clusters)
        
        draft_list_of_cluster_lists.append(list_of_clusters)
        
    training_data.append(draft_list_of_cluster_lists)

print('first draft features: ', list(all_drafts.iloc[0]))
print('training data size: ', len(training_data))
print('upgraded first draft features: ', training_data[0])
    
    

In [ ]:
import requests
resp = requests.get("https://api.opendota.com/api/heroStats?api_key=96eb7da5-ebee-4944-a6b9-61ca1127a989")
hero_stats = resp.json()
print(hero_stats)

#Preparing features
training_data = []
for index, row in all_drafts_names.iterrows():
    draft_list_of_cluster_lists = []
    for hero_name in row:        
        list_of_clusters = []
        
        #translate hero name back into id
        hero_id = name_to_hero_id_dict[hero_name]
        list_of_clusters.append(hero_id)
                
        hero_clusters = hero_clusters_dict[hero_name]
        # print("hero clusters: ", hero_clusters)
        list_of_clusters.extend(hero_clusters)
        
        for d in hero_stats:

            # Check if the current dictionary has the key-value pair 'id': 1
            if d.get('id') == hero_id:
                specific_hero_all_stats = d
                # print(d['roles'])
        # specific_hero_all_stats = hero_stats[hero_id]
        # relevant_hero_stats = []
        list_of_clusters.extend([specific_hero_all_stats['attack_type']])
        list_of_clusters.extend([specific_hero_all_stats['primary_attr']])
        list_of_clusters.extend(specific_hero_all_stats['roles'])
        #specific_hero_all_stats[], specific_hero_all_stats[], specific_hero_all_stats[], specific_hero_all_stats[],
        #specific_hero_all_stats[], specific_hero_all_stats[], specific_hero_all_stats[], specific_hero_all_stats[])
        # print("relevant hero stats: ", relevant_hero_stats)
        # list_of_clusters.extend(relevant_hero_stats)
        
        draft_list_of_cluster_lists.append(list_of_clusters)
        
    training_data.append(draft_list_of_cluster_lists)

print('first draft features: ', list(all_drafts.iloc[0]))
print('training data size: ', len(training_data))
print('upgraded first draft features: ', training_data[0])

In [ ]:
import numpy as np

# Initialize a dictionary to store the mapping of each unique categorical value to an integer index
categorical_value_to_index = {}

# Initialize a variable to store the maximum categorical length
max_categorical_length = 0

# Iterate over all data points to build the mapping dictionary and find the maximum categorical length
for data_point in training_data:
    for feature_list in data_point:
        categorical_features = np.array(feature_list)[7:]
        max_categorical_length = max(max_categorical_length, len(categorical_features))
        for value in categorical_features:
            if value not in categorical_value_to_index:
                categorical_value_to_index[value] = len(categorical_value_to_index)

# Initialize lists to store encoded categorical features and numerical features for each data point
encoded_features_list = []
numerical_features_list = []

# Iterate over each data point
for data_point in training_data:
    # Initialize lists to store encoded categorical features and numerical features for this data point
    encoded_features_data_point = []
    numerical_features_data_point = []
    
    # Iterate over each list in the data point
    for feature_list in data_point:
        # Extract numerical features
        numerical_features = np.array(feature_list)[:7]
        
        # Append the numerical features for this list to the data point list
        numerical_features_data_point.append(numerical_features)
        
        # Extract categorical features
        categorical_features = np.array(feature_list)[7:]
        
        # Convert categorical features to integers representing the index of each unique value
        encoded_features = np.array([categorical_value_to_index.get(value, 0) for value in categorical_features])
        
        # Pad the encoded features with zeros to match the maximum categorical length
        padded_encoded_features = np.pad(encoded_features, (0, max_categorical_length - len(categorical_features)), constant_values=0)
        
        # Append the padded encoded features for this list to the data point list
        encoded_features_data_point.append(padded_encoded_features)
    
    # Stack the encoded features vertically for this data point
    encoded_features_data_point = np.vstack(encoded_features_data_point)
    
    # Stack the numerical features vertically for this data point
    numerical_features_data_point = np.vstack(numerical_features_data_point)
    
    # Append the encoded and numerical features for this data point to the respective lists
    encoded_features_list.append(encoded_features_data_point)
    numerical_features_list.append(numerical_features_data_point)

# Find the maximum number of columns for both encoded and numerical features
max_encoded_columns = max(encoded_feature.shape[1] for encoded_feature in encoded_features_list)
max_numerical_columns = max(numerical_feature.shape[1] for numerical_feature in numerical_features_list)

# Pad the encoded features with zeros to match the maximum number of columns
padded_encoded_features_list = [np.pad(encoded_feature, ((0, 0), (0, max_encoded_columns - encoded_feature.shape[1])), constant_values=0) for encoded_feature in encoded_features_list]

# Concatenate encoded and numerical features for all data points
encoded_training_data = np.concatenate((numerical_features_list, padded_encoded_features_list), axis=2)

# Print the processed data
print(encoded_training_data)
print(encoded_training_data.shape)

encoded_training_data = np.array(encoded_training_data, dtype=int)
print(encoded_training_data)

In [ ]:
print(encoded_training_data[0])

In [ ]:
print(categorical_value_to_index)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

# Assuming draft_data contains the draft data with hero clusters and game results

# Prepare the training data
# X = training_data
X = encoded_training_data #10 GROUPS of: 1 hero id, 6 clusters, 8 hero traits (including zero padding for heroes without)
y = all_wins  # Target variable (game result: 0 for loss, 1 for win)

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=3435)

# Define the neural network model
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(10, 15)),  # Flatten the input shape
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    min_delta=0.001,  # Minimum change in monitored quantity to qualify as improvement
    patience=7,  # Number of epochs with no improvement after which training will be stopped
    verbose=1,  # Print message when training stops due to early stopping
    mode='auto',  # Whether to minimize or maximize the monitored quantity ('auto' determines based on metric)
    restore_best_weights=True  # Restore model weights from the epoch with the best value of the monitored quantity
)

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Get training/validation accuracy
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Plot training/validation accuracy
plt.plot(range(1, len(train_accuracy) + 1), train_accuracy, label='Training Accuracy')
plt.plot(range(1, len(val_accuracy) + 1), val_accuracy, label='Validation Accuracy')

plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
print(encoded_training_data[0])

In [ ]:
from sklearn.model_selection import train_test_split
X = np.array(encoded_training_data)
y = np.array(all_wins)

# Split the data for each team
X_team1 = X[:, 0:5, :]
X_team2 = X[:, 5:10, :]

# Split the data into training and testing sets for each team
X_team1_train, X_team1_test, X_team2_train, X_team2_test, y_train, y_test = train_test_split(
    X_team1, X_team2, y, test_size=0.15, random_state=3435,# shuffle = True
)

print('shape of x team 1: ', X_team1_train.shape)
print('shape of x team 2: ', X_team2_train.shape)
print('shape of y: ', y_train.shape)

# Save the datasets to .npy files

np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/X_team1_train.npy', X_team1_train)
np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/X_team1_test.npy', X_team1_test)
np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/X_team2_train.npy', X_team2_train)
np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/X_team2_test.npy', X_team2_test)
np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/y_train.npy', y_train)
np.save('/Users/moengke/Desktop/CDA/DotA - Course Project/y_test.npy', y_test)
print('Files saved successfully.')

X_team1_train = 'poop'


In [ ]:
import numpy as np
base_path = '/Users/moengke/Desktop/CDA/DotA - Course Project/'

X_team1_train = np.load(base_path + 'X_team1_train.npy')
X_team1_test = np.load(base_path + 'X_team1_test.npy')
X_team2_train = np.load(base_path + 'X_team2_train.npy')
X_team2_test = np.load(base_path + 'X_team2_test.npy')
y_train = np.load(base_path + 'y_train.npy')
y_test = np.load(base_path + 'y_test.npy')

print(X_team1_train.shape)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, LSTM, Permute, Concatenate, Dense, Flatten
import tqdm
from tensorflow.keras.callbacks import Callback

class CustomVerbose(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f'Starting Epoch {epoch+1}')
        self.batch = 0

    def on_batch_end(self, batch, logs=None):
        self.batch += 1
        if self.batch % 100 == 0:  # Adjust this to change how often you see updates
            print(f'Batch {self.batch}, Loss: {logs["loss"]}')

class TQDMProgressBar(Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.epochs = getattr(self, 'epochs', 0) + 1
        self.prog_bar = tqdm.tqdm(total=100, desc=f'Epoch {epoch + 1}/{self.epochs}')

    def on_batch_end(self, batch, logs=None):
        self.prog_bar.update((1 / self.params['steps']) * 100)

    def on_epoch_end(self, epoch, logs=None):
        self.prog_bar.close()

# MLP for each team
def create_mlp():
    input_layer = Input(shape=(5, 15))  # Each sample has a shape of 5x15
    x = Flatten()(input_layer)  # Flatten the 5x15 matrix to a vector of 75 elements
    x = Dense(8, activation='relu')(x)  # First dense layer
    x = Dropout(0.25)(x)
    x = Dense(4, activation='relu')(x)  # Second dense layer
    output = Dense(1, activation='sigmoid')(x)  # Output layer for binary classification
    return Model(inputs=input_layer, outputs=output)
# MLP for both teams combined
def create_combined_mlp():
    # The shape (10, 15) reflects the concatenated teams data:
    # 10 entities (e.g., heroes) each with 15 features
    input_layer = Input(shape=(10, 15))
    x = Flatten()(input_layer)  # Flatten the 10x15 matrix to a vector of 150 elements
    x = Dense(8, activation='relu')(x)  # First dense layer with more neurons
    x = Dropout(0.25)(x)
    x = Dense(4, activation='relu')(x)  # Second dense layer
    return Model(inputs=input_layer, outputs=x)


base_path = '/Users/moengke/Desktop/CDA/DotA - Course Project/'
X_team1_train = np.load(base_path + 'X_team1_train.npy')
X_team1_test = np.load(base_path + 'X_team1_test.npy')
X_team2_train = np.load(base_path + 'X_team2_train.npy')
X_team2_test = np.load(base_path + 'X_team2_test.npy')
y_train = np.load(base_path + 'y_train.npy')
y_test = np.load(base_path + 'y_test.npy')
# Combine the team data for the dual team MLP
X_combined_train = np.concatenate([X_team1_train, X_team2_train], axis=1)
X_combined_test = np.concatenate([X_team1_test, X_team2_test], axis=1)
print(X_team1_train.shape)


# Instantiate MLPs
team1_mlp = create_mlp()
team2_mlp = create_mlp()
combined_mlp = create_combined_mlp()
# Inputs
team1_input = Input(shape=(5, 15))
team2_input = Input(shape=(5, 15))
combined_input = Input(shape=(10, 15))
# Get MLP outputs
team1_output = team1_mlp(team1_input)
team2_output = team2_mlp(team2_input)
combined_output = combined_mlp(combined_input)
# Concatenate all MLP outputs
concatenated = Concatenate()([team1_output, team2_output, combined_output])
# Final MLP for prediction
x = Dense(8, activation='relu')(concatenated)
x = Dense(4, activation='relu')(x)
final_output = Dense(1, activation='sigmoid')(x)
# Final model
model = Model(inputs=[team1_input, team2_input, combined_input], outputs=final_output)


# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=4, verbose=1, mode='auto', restore_best_weights=True)
# Train the model
history = model.fit(
    [X_team1_train, X_team2_train, X_combined_train],
    y_train,
    validation_split=0.1,  # Using 10% of the training data for validation
    epochs=50,
    #batch_size=4,
    callbacks=[early_stopping, TQDMProgressBar(), CustomVerbose()]
)
# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    [X_team1_test, X_team2_test, X_combined_test],
    y_test
)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Concatenate, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from spektral.layers import GCNConv
from tensorflow.keras.callbacks import EarlyStopping

# Load the data
base_path = '/Users/moengke/Desktop/CDA/DotA - Course Project/'
X_team1_train = np.load(base_path + 'X_team1_train.npy')
X_team1_test = np.load(base_path + 'X_team1_test.npy')
X_team2_train = np.load(base_path + 'X_team2_train.npy')
X_team2_test = np.load(base_path + 'X_team2_test.npy')
y_train = np.load(base_path + 'y_train.npy')
y_test = np.load(base_path + 'y_test.npy')

# Define the model
def create_gnn_branch(input_shape, adjacency_input, features_input, output_units=32):
    feat_input = Input(shape=(None, input_shape), name=f'features_input_{features_input}')
    adj_input = Input(shape=(None, None), sparse=True, name=f'adjacency_input_{adjacency_input}')
    gc1 = GCNConv(64, activation='relu')([feat_input, adj_input])
    gc2 = GCNConv(output_units, activation='relu')([gc1, adj_input])
    pool = GlobalAveragePooling1D()(gc2)
    return Model(inputs=[feat_input, adj_input], outputs=pool, name=f'gnn_branch_{features_input}')

def create_full_connected_adjacency(num_nodes):
    """Create a fully connected adjacency matrix."""
    # Creating a matrix where each node is connected to every other node
    adjacency = np.ones((num_nodes, num_nodes)) - np.eye(num_nodes)
    return adjacency

# Number of heroes per team
num_heroes_per_team = 5

# Create fully connected adjacency matrices for each team
A1 = create_full_connected_adjacency(num_heroes_per_team)
A2 = create_full_connected_adjacency(num_heroes_per_team)

# Create a combined fully connected adjacency matrix for all heroes (10 heroes)
num_heroes_combined = num_heroes_per_team * 2
A_combined = create_full_connected_adjacency(num_heroes_combined)

# Create GNN branches
gnn_branch_1 = create_gnn_branch(total_traits, 'A1', 'team_1')
gnn_branch_2 = create_gnn_branch(total_traits, 'A2', 'team_2')
gnn_combined = create_gnn_branch(total_traits, 'A_combined', 'combined')

# Input layers
features_team_1 = Input(shape=(num_heroes_per_team, total_traits), name='features_team_1')
features_team_2 = Input(shape=(num_heroes_per_team, total_traits), name='features_team_2')
features_combined = Input(shape=(num_heroes_per_team * 2, total_traits), name='features_combined')

# Get outputs from GNN branches
output_1 = gnn_branch_1([features_team_1, A1])
output_2 = gnn_branch_2([features_team_2, A2])
output_combined = gnn_combined([features_combined, A_combined])

# Combine outputs
combined_output = Concatenate()([output_1, output_2, output_combined])

# MLP for final processing
mlp_layer = Dense(64, activation='relu')(combined_output)
final_output = Dense(1, activation='sigmoid')(mlp_layer)

# Final model
model = Model(inputs=[features_team_1, features_team_2, features_combined, A1, A2, A_combined], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Split training data to create a validation set
validation_split = 0.1
val_size = int(len(X_team1_train) * validation_split)
X_team1_val, X_team2_val, y_val = X_team1_train[:val_size], X_team2_train[:val_size], y_train[:val_size]
X_team1_train, X_team2_train, y_train = X_team1_train[val_size:], X_team2_train[val_size:], y_train[val_size:]

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=4, verbose=1, mode='auto', restore_best_weights=True)

# Train the model
history = model.fit(
    [X_team1_train, X_team2_train, np.concatenate([X_team1_train, X_team2_train], axis=1), A1, A2, A_combined],
    y_train,
    validation_data=([X_team1_val, X_team2_val, np.concatenate([X_team1_val, X_team2_val], axis=1), A1, A2, A_combined], y_val),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    [X_team1_test, X_team2_test, np.concatenate([X_team1_test, X_team2_test], axis=1), A1, A2, A_combined],
    y_test
)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
import tensorflow as tf

# Try importing various keras modules to check their existence
try:
    from tensorflow.keras import models
    from tensorflow.keras import layers
    print("Keras modules are importable.")
except ImportError as e:
    print("Error importing Keras:", e)

# Specifically check for datasets
try:
    from tensorflow.keras.datasets import mnist
    print("MNIST module is importable.")
except ImportError as e:
    print("Error importing MNIST:", e)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Input, LSTM, Permute, Concatenate, Dense, Flatten
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Assuming encoded_training_data and all_wins are defined
# ...

X = np.array(encoded_training_data)
y = np.array(all_wins)

# Split the data for each team
X_team1 = X[:, 0:5, :]
X_team2 = X[:, 5:10, :]

# Split the data into training and testing sets for each team
X_team1_train, X_team1_test, X_team2_train, X_team2_test, y_train, y_test = train_test_split(
    X_team1, X_team2, y, test_size=0.15, random_state=3435,# shuffle = True
)

print('size of x team 1: ', X_team1_train.size)
print('size of x team 2: ', X_team2_train.size)
print('size of y: ', y_train.size)


# Model architecture
input_1 = Input(shape=(5, 15))  # Input for team 1
input_2 = Input(shape=(5, 15))  # Input for team 2

lstm_1 = LSTM(150, return_sequences=True, activation='tanh')(input_1)
permute_1 = Permute((2, 1))(lstm_1)

lstm_2 = LSTM(150, return_sequences=True, activation='tanh')(input_2)
permute_2 = Permute((2, 1))(lstm_2)

# Concatenate the outputs from both teams
concatenated = Concatenate()([permute_1, permute_2])

# Flatten the concatenated features
flatten = Flatten()(concatenated)

# Additional dense layers as in your original model
dense_output = Dense(256, activation='relu')(flatten)
dense_output = Dense(128, activation='relu')(dense_output)
dense_output = Dense(64, activation='relu')(dense_output)
output = Dense(1, activation='sigmoid')(dense_output)

model = keras.Model(inputs=[input_1, input_2], outputs=output)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping callback
early_stopping = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=4,
    verbose=1,
    mode='auto',
    restore_best_weights=True
)

# Train the model
history = model.fit(
    [X_team1_train, X_team2_train],
    y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stopping]
)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(
    [X_team1_test, X_team2_test],
    y_test
)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

# Plot training and validation accuracy
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Print the model summary to understand its layers
model.summary()

# If you have pydot and graphviz installed, you can also plot the model as a graph
# This will not execute in this environment but can be run in a local environment with the necessary libraries
try:
    # Save the model plot to a file
    tf.keras.utils.plot_model(model, to_file='/mnt/data/model_plot.png', show_shapes=True, show_layer_names=True)
    print("Model plot saved.")
except ImportError as e:
    print("pydot or graphviz libraries are not available to plot the model.")

# To display the saved image if it was successful
from IPython.display import Image
Image(filename='/mnt/data/model_plot.png')


# Assuming you have a single match input split into team1_data and team2_data
team1_data = X[0:1, 0:5, :]  # Data for the first team of the first match
team2_data = X[0:1, 5:10, :] # Data for the second team of the first match

# Pass the data through the model to simulate inference
output = model.predict([team1_data, team2_data])

# Describe the flow through the network
flow_description = f"""
The input data for the match is split into two arrays: one for each team.
- Team 1 data shape: {team1_data.shape}
- Team 2 data shape: {team2_data.shape}

This data is then fed into their respective LSTM layers, which process the sequences.
The outputs from the LSTMs are then permuted and concatenated to form a single array, which is then flattened.

The flattened array is passed through a series of Dense layers with ReLU activations, and finally, a Dense layer with a sigmoid activation function to predict the match outcome.

The model output (probability of a win for team 1) is: {output[0][0]}
"""

print(flow_description)

